Initiales setup für GPU Verwendung

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Installieren der Dependencies

In [ ]:
!pip install spacy==3.3.1 
!pip install spacy-transformers
!pip install -U spacy[transformers,cuda113]
#for transformers v4.0.0+ and models that require SentencePiece (e.g., ALBERT, CamemBERT, XLNet, Marian, and T5)
!pip install transformers[sentencepiece]
!pip install spacy-sentence-bert

In [ ]:
!pip install shap

Laden der Spacy Modelle

In [ ]:
!python -m spacy download de_core_news_lg
!python -m spacy download de_dep_news_trf